## PSORTb Result Scraping Script

**This script automates the process of sending protein sequence data to the PSORTb database (https://db.psort.org/search/blast) and retrieving the prediction results. The input is a FASTA file containing protein sequences. For each sequence, the script sends a HTTP POST request to the PSORTb database with the protein sequence data. There are multiple hits for each sequence with different identity percentage. This code parses all the hits to extract relevant information.**

**The relevant information extracted includes:**

- Sequence description
- Subcellular localization prediction
- Bit score
- E-value
- Percent identity

**This information is stored in a DataFrame, and then saved to a csv file for further analysis**

**This database is slightly different then (https://www.psort.org/psortb/index.html), as this provides matched hits using blast algorithm. And the other one provides summerized result for input sequence**

In [10]:
"""
Zaidur Rahman
May 27, 2023
Description: This script scrape PSORTb blast result from website

"""

import requests
from Bio import SeqIO
import re
import pandas as pd


url = 'https://db.psort.org/search/blast'

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://db.psort.org',
    'referer': 'https://db.psort.org/search/blast'
}

fasta_file = "/Users/zaidur/Documents/Sequence_Project/aeromonasBact/test_fasta.fasta" 
sequences = list(SeqIO.parse(fasta_file, "fasta"))

# Initialize an empty list to store our rows
rows = []

for sequence in sequences:
    data ={ 
        'filter': '1',
        'evalue': '30',
        'database': 'c',
        'blast': 'blastp',
        'sequences': f'>{sequence.id}\n{sequence.seq}'
    }

    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()

        text = response.text
        lines = text.split('\n')

        for i, line in enumerate(lines):
            if line.startswith('>'):
                full_line = lines[i]+lines[i+1]+lines[i+2]
                parts = full_line.split('|')
                if len(parts) < 5:
                    continue
                row = {
                    'sequence_id': sequence.id,
                    'description': '|'.join(parts[:3] + parts[4:]),
                    'prediction': parts[3]
                }

            elif line.startswith(' Score = '):
                full_line = lines[i]+lines[i+1]
                bit_score = re.search(r'Score = ([\d\.]+)', full_line)
                evalue = re.search(r'Expect = ([\de\.-]+)', full_line)
                percent_identity = re.search(r'Identities = \d+/\d+ \(([\d\.]+)%\)', full_line)
                if bit_score and evalue and percent_identity:
                    row.update({
                        'bit_score': bit_score.group(1),
                        'evalue': evalue.group(1),
                        'percent_identity': percent_identity.group(1),
                    })
                    rows.append(row)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# Create a DataFrame from our list of rows
df = pd.DataFrame(rows)
df

,sequence_id,description,prediction,bit_score,evalue,percent_identity
0,A0A068FVC1,> gnl|PSORTDB|WP_101613939.1-NZ_CP028568.1:c34...,Unknown,341,2e-119,99
1,A0A068FVC1,> gnl|PSORTDB|WP_103261933.1-NZ_CP026226.1:c43...,Unknown,259,1e-86,80
2,A0A068FVC1,> gnl|PSORTDB|WP_113070000.1-NZ_CP025705.1:293...,Unknown,256,2e-85,78
3,A0A068FVC1,> gnl|PSORTDB|WP_106842527.1-NZ_CP015448.1:c38...,Unknown,254,6e-85,76
4,A0A068FVC1,> gnl|PSORTDB|WP_045790994.1-NZ_CP011100.1:157...,Unknown,254,1e-84,79
...,...,...,...,...,...,...
1995,A0A068FZD0,> gnl|PSORTDB|WP_070237321.1-NZ_CP017478.1:246...,Cytoplasmic,353,4e-119,50
1996,A0A068FZD0,> gnl|PSORTDB|WP_130410225.1-NZ_CP022305.1:391...,Cytoplasmic,352,4e-119,54
1997,A0A068FZD0,> gnl|PSORTDB|WP_129440682.1-NZ_CP035492.1:c22...,Cytoplasmic,352,5e-119,55
1998,A0A068FZD0,> gnl|PSORTDB|WP_011538816.1-NC_008044.1:c1582...,Cytoplasmic,352,5e-119,52


In [14]:
"""
The version is without exception handling

"""

import requests
from Bio import SeqIO
import re
import pandas as pd

url = 'https://db.psort.org/search/blast'

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://db.psort.org',
    'referer': 'https://db.psort.org/search/blast'
}

fasta_file = "/Users/zaidur/Documents/Sequence_Project/aeromonasBact/test_fasta.fasta"  # replace with your fasta file name
sequences = list(SeqIO.parse(fasta_file, "fasta"))

# Initialize an empty list to store our rows
rows = []

for sequence in sequences:
    data ={ 
        'filter': '1',
        'evalue': '30',
        'database': 'c',
        'blast': 'blastp',
        'sequences': f'>{sequence.id}\n{sequence.seq}'
    }

    response = requests.post(url, headers=headers, data=data)
    text = response.text
    lines = text.split('\n')

    for i, line in enumerate(lines):
        if line.startswith('>'):
            full_line = lines[i]+lines[i+1]+lines[i+2]
            parts = full_line.split('|')
            if len(parts) < 5:
                continue
            row = {
                'sequence_id': sequence.id,
                'description': '|'.join(parts[:3] + parts[4:]),
                'prediction': parts[3]
            }

        elif line.startswith(' Score = '):
            full_line = lines[i]+lines[i+1]
            bit_score = re.search(r'Score = ([\d\.]+)', full_line)
            evalue = re.search(r'Expect = ([\de\.-]+)', full_line)
            percent_identity = re.search(r'Identities = \d+/\d+ \(([\d\.]+)%\)', full_line)
            if bit_score and evalue and percent_identity:
                row.update({
                    'bit_score': bit_score.group(1),
                    'evalue': evalue.group(1),
                    'percent_identity': percent_identity.group(1),
                })
                rows.append(row)

# Create a DataFrame from our list of rows
df = pd.DataFrame(rows)
df

,sequence_id,description,prediction,bit_score,evalue,percent_identity
0,A0A068FVC1,> gnl|PSORTDB|WP_101613939.1-NZ_CP028568.1:c34...,Unknown,341,2e-119,99
1,A0A068FVC1,> gnl|PSORTDB|WP_103261933.1-NZ_CP026226.1:c43...,Unknown,259,1e-86,80
2,A0A068FVC1,> gnl|PSORTDB|WP_113070000.1-NZ_CP025705.1:293...,Unknown,256,2e-85,78
3,A0A068FVC1,> gnl|PSORTDB|WP_106842527.1-NZ_CP015448.1:c38...,Unknown,254,6e-85,76
4,A0A068FVC1,> gnl|PSORTDB|WP_045790994.1-NZ_CP011100.1:157...,Unknown,254,1e-84,79
...,...,...,...,...,...,...
1995,A0A068FZD0,> gnl|PSORTDB|WP_070237321.1-NZ_CP017478.1:246...,Cytoplasmic,353,4e-119,50
1996,A0A068FZD0,> gnl|PSORTDB|WP_130410225.1-NZ_CP022305.1:391...,Cytoplasmic,352,4e-119,54
1997,A0A068FZD0,> gnl|PSORTDB|WP_129440682.1-NZ_CP035492.1:c22...,Cytoplasmic,352,5e-119,55
1998,A0A068FZD0,> gnl|PSORTDB|WP_011538816.1-NC_008044.1:c1582...,Cytoplasmic,352,5e-119,52


In [16]:
"""
This is version uses multiprocess module for parallel processing. This was written for HPC cluster
The multiprocessing Pool will create a pool of worker processes. It then uses these processes to execute
the function process_sequence in parallel for each element in sequences.

In this script, the process_sequence function is processing each sequence in the input and recording all
hits for that sequence. The result of this function is a list of rows, where each row corresponds to a different hit for the sequence.

Then, in the main part of the script, we're using a multiprocessing Pool to process each sequence in parallel.
The result is a list of results, where each result is a list of rows for a different sequence. We then flatten
this list of lists into a single list of rows, which we use to create the dataframe.

Jupyter notebook can't handle the multiprocess module well. This code needs to be run as a python script.

"""



import requests
from Bio import SeqIO
import re
import pandas as pd
from multiprocessing import Pool

url = 'https://db.psort.org/search/blast'

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://db.psort.org',
    'referer': 'https://db.psort.org/search/blast'
}

fasta_file = "/Users/zaidur/Documents/Sequence_Project/aeromonasBact/test_fasta.fasta" 
sequences = list(SeqIO.parse(fasta_file, "fasta"))

def process_sequence(sequence):
    data ={ 
        'filter': '1',
        'evalue': '30',
        'database': 'c',
        'blast': 'blastp',
        'sequences': f'>{sequence.id}\n{sequence.seq}'
    }

    rows = []
    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()

        text = response.text
        lines = text.split('\n')
        row = {}

        for i, line in enumerate(lines):
            if line.startswith('>'):
                if row:  # if row is not empty, append it to rows
                    rows.append(row)
                full_line = lines[i]+lines[i+1]+lines[i+2]
                parts = full_line.split('|')
                if len(parts) < 5:
                    continue
                row = {
                    'sequence_id': sequence.id,
                    'description': '|'.join(parts[:3] + parts[4:]),
                    'prediction': parts[3]
                }

            elif line.startswith(' Score = '):
                full_line = lines[i]+lines[i+1]
                bit_score = re.search(r'Score = ([\d\.]+)', full_line)
                evalue = re.search(r'Expect = ([\de\.-]+)', full_line)
                percent_identity = re.search(r'Identities = \d+/\d+ \(([\d\.]+)%\)', full_line)
                if bit_score and evalue and percent_identity:
                    row.update({
                        'bit_score': bit_score.group(1),
                        'evalue': evalue.group(1),
                        'percent_identity': percent_identity.group(1),
                    })

        # Add the last row
        if row:
            rows.append(row)
        
        return rows

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    with Pool() as p:
        results = p.map(process_sequence, sequences)
    rows = [row for result in results for row in result]
    df = pd.DataFrame(rows)
    df.to_csv('psortb_blast_test_output.csv', index=False)

Process SpawnPoolWorker-13:
Process SpawnPoolWorker-11:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/zaidur/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/zaidur/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/zaidur/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zaidur/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/zaidur/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zaidur/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
  File "/Users/zaidur/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = g

KeyboardInterrupt: 